In [1]:
# import required libs
import glob
import os
import sys

import tensorflow as tf
import tensorflow_data_validation as tfdv

target_dir = os.path.dirname(os.getcwd())
target_dir_fraud = target_dir + "/fraud"
sys.path.extend([target_dir, target_dir_fraud])

In [2]:
# Read artifact information from metadata store.
from fraud.local_runner import METADATA_PATH

from tfx.orchestration import metadata
from tfx.types import standard_artifacts

metadata_connection_config = metadata.sqlite_metadata_connection_config(METADATA_PATH)
with metadata.Metadata(metadata_connection_config) as store:
    stats_artifacts = store.get_artifacts_by_type(standard_artifacts.ExampleStatistics.TYPE_NAME)
    schema_artifacts = store.get_artifacts_by_type(standard_artifacts.Schema.TYPE_NAME)
    anomalies_artifacts = store.get_artifacts_by_type(standard_artifacts.ExampleAnomalies.TYPE_NAME)


In [3]:
# configure output paths
# Exact paths to output artifacts can also be found on KFP Web UI if you are using kubeflow.
stats_path = stats_artifacts[-1].uri
stats_file = os.path.join(stats_path, 'FeatureStats.pb')
train_stats_file = os.path.join(stats_path, 'train', 'stats_tfrecord')
eval_stats_file = os.path.join(stats_path, 'eval', 'stats_tfrecord')
print("Train stats file:{}, Eval stats file:{}".format(train_stats_file, eval_stats_file))

schema_file = os.path.join(schema_artifacts[-1].uri, 'schema.pbtxt')
print("Generated schame file:{}".format(schema_file))
anomalies_file = os.path.join(anomalies_artifacts[-1].uri, 'anomalies.pbtxt')
print("Generated anomalies file:{}".format(anomalies_file))

Train stats file:/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/tfx_pipeline_output/fraud/Transform/post_transform_stats/78/train/stats_tfrecord, Eval stats file:/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/tfx_pipeline_output/fraud/Transform/post_transform_stats/78/eval/stats_tfrecord
Generated schame file:/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/tfx_pipeline_output/fraud/Transform/pre_transform_schema/78/schema.pbtxt
Generated anomalies file:/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/tfx_pipeline_output/fraud/Transform/post_transform_anomalies/78/anomalies.pbtxt


In [14]:
# load generated statistics from StatisticsGen
stats = tfdv.load_statistics(stats_file)
#train_stats = tfdv.load_statistics(train_stats_file)
#eval_stats = tfdv.load_statistics(eval_stats_file)
tfdv.visualize_statistics(
    lhs_statistics=stats_file,
    rhs_statistics=None,
    lhs_name='DATASET',
)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xcf in position 1: invalid continuation byte

In [6]:
# load generated schema from SchemaGen
schema = tfdv.load_schema_text(schema_file)
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'NAME_CONTRACT_TYPE',STRING,required,,'NAME_CONTRACT_TYPE'
'NAME_TYPE_SUITE',STRING,required,,'NAME_TYPE_SUITE'
'AMT_CREDIT',FLOAT,required,,-
'AMT_INCOME_TOTAL',FLOAT,required,,-
'CNT_CHILDREN',INT,required,,-
'FLAG_MOBIL',INT,required,,-
'TARGET',INT,required,,-


/Users/ivan/.pyenv/versions/3.8.12/envs/py-tfx-38/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'NAME_CONTRACT_TYPE',"'Cash loans', 'Revolving loans'"
'NAME_TYPE_SUITE',"'Children', 'Family', 'Group of people', 'Other_A', 'Other_B', 'Spouse, partner', 'Unaccompanied'"


In [15]:
# load data validation result from ExampleValidator
#anomalies = tfdv.load_anomalies_text(anomalies_file)
#tfdv.display_anomalies(anomalies)